## Use Gen/AI to prepare CV

### libraries
use GPU!!! may be google colab!

In [ ]:
install_libraries = True

# Toggle this to True when running in Colab
USE_COLAB = False


if install_libraries:
    # --- Base Python Requirements ---
    !pip install pathlib
    !pip install openai
    !pip install anthropic
    !pip install ollama
    !pip install transformers torch accelerate

    # --- ⚠️ CRITICAL: Install Ollama Server Application ---
    # This downloads and installs the *Ollama executable* on the Colab VM.
    print("\nInstalling Ollama Server Application...")
    !curl -fsSL https://ollama.com/install.sh | sh

    # --- ❌ REMOVE THIS LINE ---
    # !ollama pull llama3.1:8b
    # This must be done *after* the server is started in the next step.


Installing Ollama Server Application...
ERROR: This script is intended to run on Linux only.


In [ ]:

import warnings
warnings.filterwarnings('ignore')

import os
import re
import json
import glob
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from datetime import datetime
from collections import defaultdict


# ═══════════════════════════════════════════════════════════════════
# 📄 AI-POWERED CV CUSTOMIZER V2
# Uses organized/deduplicated CV files as reference
# Supports multiple LLM providers (OpenAI, Anthropic, Ollama, HuggingFace)
# CPU & GPU Compatible
# ═══════════════════════════════════════════════════════════════════

import warnings
warnings.filterwarnings('ignore')



## TODO:   from langchain_community.document_loaders import WebBaseLoader



In [ ]:
import os
import subprocess
import time

def setup_ollama_in_colab(model_name: str = 'llama3.1:8b'):
    """Starts the Ollama server and pulls the required model."""

    print("\n--- 🛠️ Starting Ollama Server and Pulling Model ---")

    # 1. Set Environment Variable and Start the Ollama Service
    ollama_host = '127.0.0.1:11434'
    os.environ['OLLAMA_HOST'] = ollama_host
    print(f"1. Setting OLLAMA_HOST environment variable to: {ollama_host}")

    # Start the Ollama server process in the background
    print("2. Starting 'ollama serve' in background...")
    try:
        # Popen is critical here to keep the process running after this cell finishes
        subprocess.Popen(['ollama', 'serve'], start_new_session=True)
        time.sleep(5) # Wait for the server to start
        print("   -> Server started successfully on 127.0.0.1:11434.")
    except Exception as e:
        print(f"❌ ERROR: Failed to start server: {e}")
        return

    # 3. Pull the Model
    print(f"3. Pulling model: {model_name}. This may take some time...")
    try:
        subprocess.run(
            ['ollama', 'pull', model_name],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print(f"✅ Successfully pulled model: {model_name}")

    except subprocess.CalledProcessError as e:
        print(f"❌ ERROR: Failed to pull model {model_name}. Check model name and internet connection.")
        print(f"   Stderr: {e.stderr.strip()}")

    print("--- ✅ Ollama Setup Complete ---")

In [ ]:
import os

# User selects LLM provider explicitly
LLM_PROVIDER = input("Select LLM provider (openai, anthropic, ollama, huggingface): ").strip().lower()

# Set environment variable for consistency (optional)
os.environ['LLM_PROVIDER'] = LLM_PROVIDER


if USE_COLAB:
    from google.colab import drive

    drive.mount('/content/drive')

    # Colab-specific paths
    ORGANIZED_FILES_PATH = "/content/drive/MyDrive/Documents/work/jobs_work_applications_and_supporting_docs/AI/CVS_awesome_files/"
    OUTPUT_BASE_PATH = "/content/drive/MyDrive/Documents/work/jobs_work_applications_and_supporting_docs/AI/output/"

    import os
    # List files in the folder
    for filename in os.listdir(ORGANIZED_FILES_PATH):
      print(filename)


    # Prompt for API keys
    openai_key = input("Enter your OpenAI API key (leave blank if not using): ").strip()
    anthropic_key = input("Enter your Anthropic API key (leave blank if not using): ").strip()
    huggingface_key = input("Enter your HuggingFace API key (leave blank if not using): ").strip()

    # Set environment variables
    os.environ['OPENAI_API_KEY'] = openai_key
    os.environ['ANTHROPIC_API_KEY'] = anthropic_key
    os.environ['HUGGINGFACE_API_KEY'] = huggingface_key

    if LLM_PROVIDER == "ollama":
        # --- EXECUTE THE FUNCTION ---
        # Pass the model name you need from your Config
        setup_ollama_in_colab(model_name='llama3.1:8b')

else:
    # Local paths
    ORGANIZED_FILES_PATH = "/Users/kbillis/bin/Awesome-CV/organized_filesdeduplicated"
    OUTPUT_BASE_PATH = "/Users/kbillis/bin/Awesome-CV/examples/AI/"


# Unified config class
class Config:
    """Configuration for CV generation."""

    ORGANIZED_FILES_PATH = ORGANIZED_FILES_PATH
    OUTPUT_BASE_PATH = OUTPUT_BASE_PATH

    LLM_PROVIDER = LLM_PROVIDER

    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', '')
    ANTHROPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY', '')
    HUGGINGFACE_API_KEY = os.environ.get('HUGGINGFACE_API_KEY', '')

    OPENAI_MODEL = 'gpt-4'
    ANTHROPIC_MODEL = 'claude-3-5-sonnet-20241022'
    OLLAMA_MODEL = 'llama3.1:8b'
    HUGGINGFACE_MODEL = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

    USE_GPU = True
    MAX_TOKENS = 2000
    TEMPERATURE = 0.3

    CV_CATEGORIES = ['summary', 'experience', 'key_skills', 'education', 'additional_info']



In [ ]:
# @title
# test your openai key
# import os
# from openai import OpenAI

# client = OpenAI(
    # This is the default and can be omitted
#      api_key=openai_key,
# )

# response = client.responses.create(
#     model="gpt-4o",
#     instructions="You are a coding assistant that talks like a pirate.",
#     input="How do I check if a Python object is an instance of a class?",
# )

# print(response.output_text)

In [ ]:


# ═══════════════════════════════════════════════════════════════════
# LLM PROVIDERS (Same as before - reusing)
# ═══════════════════════════════════════════════════════════════════

class LLMProvider:
    """Base class for LLM providers."""
    def generate(self, prompt: str) -> str:
        raise NotImplementedError

class OpenAIProvider(LLMProvider):
    def __init__(self, config: Config):
        self.config = config
        try:
            from openai import OpenAI
            self.client = OpenAI(api_key=config.OPENAI_API_KEY)
            print(f"✅ OpenAI initialized: {config.OPENAI_MODEL}")
        except ImportError:
            print("❌ pip install openai")
            self.client = None

    def generate(self, prompt: str) -> str:
        if not self.client:
            return "Error: OpenAI not initialized"
        try:
            response = self.client.chat.completions.create(
                model=self.config.OPENAI_MODEL,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=self.config.MAX_TOKENS,
                temperature=self.config.TEMPERATURE
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error: {e}"

class AnthropicProvider(LLMProvider):
    def __init__(self, config: Config):
        self.config = config
        try:
            from anthropic import Anthropic
            self.client = Anthropic(api_key=config.ANTHROPIC_API_KEY)
            print(f"✅ Anthropic initialized: {config.ANTHROPIC_MODEL}")
        except ImportError:
            print("❌ pip install anthropic")
            self.client = None

    def generate(self, prompt: str) -> str:
        if not self.client:
            return "Error: Anthropic not initialized"
        try:
            message = self.client.messages.create(
                model=self.config.ANTHROPIC_MODEL,
                max_tokens=self.config.MAX_TOKENS,
                temperature=self.config.TEMPERATURE,
                messages=[{"role": "user", "content": prompt}]
            )
            return message.content[0].text
        except Exception as e:
            return f"Error: {e}"


class OllamaProvider(LLMProvider):
    def __init__(self, config: Config):
        self.config = config

        # 1. Define the local URL (Ollama's default)
        # You can add this to your Config class for consistency if you want,
        # but the default is usually fine for local use.
        self.base_url = os.environ.get('OLLAMA_HOST', 'http://127.0.0.1:11434')

        try:
            from ollama import Client # <-- Import the Client class

            # 2. Instantiate the Client explicitly with the host URL
            self.client = Client(host=self.base_url)

            # 3. Perform a health check/list models using the Client instance
            # Note: client.list() is the correct method for the instantiated Client
            models = self.client.list()

            print(f"✅ Ollama initialized on {self.base_url} with model: {config.OLLAMA_MODEL}")

        except ImportError:
            print("❌ pip install ollama")
            self.client = None
        except requests.exceptions.ConnectionError:
            # Catch the error if the server is not actually running at the host
            print(f"❌ ConnectionError: Ollama server not running at {self.base_url}")
            self.client = None
        except Exception as e:
            # Catch other potential errors (like no models pulled)
            print(f"❌ Error initializing Ollama client: {e}")
            self.client = None

    def generate(self, prompt: str) -> str:
        if not self.client:
            return "Error: Ollama not initialized"
        try:
            # Use the client instance's generate method
            response = self.client.generate(
                model=self.config.OLLAMA_MODEL,
                prompt=prompt,
                options={
                    'temperature': self.config.TEMPERATURE,
                    'num_predict': self.config.MAX_TOKENS,
                }
            )
            return response['response']
        except Exception as e:
            return f"Error during generation: {e}"


class OllamaProvider_local(LLMProvider):
    def __init__(self, config: Config):
        self.config = config
        try:
            import ollama
            self.client = ollama
            models = self.client.list()
            print(f"✅ Ollama initialized: {config.OLLAMA_MODEL}")
        except ImportError:
            print("❌ pip install ollama")
            self.client = None

    def generate(self, prompt: str) -> str:
        if not self.client:
            return "Error: Ollama not initialized"
        try:
            response = self.client.generate(
                model=self.config.OLLAMA_MODEL,
                prompt=prompt,
                options={
                    'temperature': self.config.TEMPERATURE,
                    'num_predict': self.config.MAX_TOKENS,
                }
            )
            return response['response']
        except Exception as e:
            return f"Error: {e}"

class HuggingFaceProvider(LLMProvider):
    def __init__(self, config: Config):
        self.config = config
        try:
            import torch
            from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

            device = "cuda" if torch.cuda.is_available() and config.USE_GPU else "cpu"
            print(f"🖥️  Device: {device}")
            print(f"📥 Loading {config.HUGGINGFACE_MODEL}...")

            self.tokenizer = AutoTokenizer.from_pretrained(config.HUGGINGFACE_MODEL)
            self.model = AutoModelForCausalLM.from_pretrained(
                config.HUGGINGFACE_MODEL,
                torch_dtype=torch.float16 if device == "cuda" else torch.float32,
                device_map="auto" if device == "cuda" else None,
                low_cpu_mem_usage=True
            )

            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                device=0 if device == "cuda" else -1
            )
            print(f"✅ HuggingFace loaded on {device}")
        except ImportError:
            print("❌ pip install transformers torch")
            self.model = None

    def generate(self, prompt: str) -> str:
        if not self.model:
            return "Error: HuggingFace not initialized"
        try:
            outputs = self.pipeline(
                prompt,
                max_new_tokens=self.config.MAX_TOKENS,
                temperature=self.config.TEMPERATURE,
                do_sample=True,
                top_p=0.9
            )
            return outputs[0]['generated_text'][len(prompt):]
        except Exception as e:
            return f"Error: {e}"

def get_llm_provider(config: Config) -> LLMProvider:
    """Factory function to get LLM provider."""
    provider = config.LLM_PROVIDER.lower()

    if provider == 'openai':
        return OpenAIProvider(config)
    elif provider == 'anthropic':
        return AnthropicProvider(config)
    elif provider == 'ollama':
        return OllamaProvider(config)
    elif provider == 'huggingface':
        return HuggingFaceProvider(config)
    else:
        raise ValueError(f"Unknown provider: {provider}")

In [22]:


# ═══════════════════════════════════════════════════════════════════
# ORGANIZED FILES ANALYZER
# ═══════════════════════════════════════════════════════════════════

class OrganizedCVAnalyzer:
    """Analyze organized CV files by category."""

    def __init__(self, organized_path: str):
        self.organized_path = Path(organized_path)
        self.categories = {}

    def load_all_categories(self) -> Dict[str, List[Dict]]:
        """Load all CV files organized by category."""
        print("\n🔍 Loading organized CV files...")

        for category_dir in self.organized_path.iterdir():
            if not category_dir.is_dir():
                continue

            category = category_dir.name
            files = []

            for tex_file in category_dir.glob("*.tex"):
                if tex_file.name == "INDEX.md":
                    continue

                try:
                    with open(tex_file, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    # Extract source directory from filename
                    # Format: SourceDir__filename.tex
                    parts = tex_file.stem.split('__')
                    source_dir = parts[0] if len(parts) > 1 else 'unknown'
                    original_name = '__'.join(parts[1:]) if len(parts) > 1 else tex_file.stem

                    files.append({
                        'path': str(tex_file),
                        'filename': tex_file.name,
                        'source_dir': source_dir,
                        'original_name': original_name,
                        'content': content,
                        'size': len(content)
                    })
                except Exception as e:
                    print(f"   ⚠️  Error reading {tex_file}: {e}")

            self.categories[category] = files
            print(f"   ✅ {category:20s}: {len(files):3d} files")

        return self.categories

    def find_relevant_files(
        self,
        job_description: str,
        category: str,
        top_k: int = 5
    ) -> List[Dict]:
        """Find most relevant files for a category based on job description."""

        if category not in self.categories:
            print(f"⚠️  Category '{category}' not found")
            return []

        files = self.categories[category]

        # Extract keywords from job description
        keywords = self._extract_keywords(job_description)

        # Score each file
        scored_files = []
        for file_data in files:
            score = self._calculate_relevance_score(
                file_content=file_data['content'],
                keywords=keywords
            )
            scored_files.append((file_data, score))

        # Sort by score
        scored_files.sort(key=lambda x: x[1], reverse=True)

        # Return top K
        top_files = [f for f, score in scored_files[:top_k]]

        print(f"   📋 Top {len(top_files)} files for {category}:")
        for i, (file_data, score) in enumerate(scored_files[:top_k], 1):
            print(f"      {i}. {file_data['source_dir']:30s} (score: {score})")

        return top_files

    def _extract_keywords(self, text: str) -> List[str]:
        """Extract keywords from text."""
        text = text.lower()

        # Domain-specific keywords
        keywords = [
            # Bioinformatics
            'bioinformatics', 'genomics', 'rna', 'dna', 'sequencing', 'ngs',
            'cancer', 'biomarker', 'omics', 'proteomics', 'transcriptomics',
            'single-cell', 'bulk-rna', 'chip-seq', 'atac-seq',

            # Data Science
            'data science', 'machine learning', 'deep learning', 'ai',
            'statistical', 'modeling', 'prediction', 'classification',
            'clustering', 'nlp', 'computer vision',

            # Programming
            'python', 'r', 'java', 'scala', 'c++', 'julia',
            'sql', 'bash', 'perl', 'javascript',

            # Tools & Platforms
            'aws', 'azure', 'gcp', 'cloud', 'docker', 'kubernetes',
            'spark', 'hadoop', 'kafka', 'airflow', 'mlflow',
            'tensorflow', 'pytorch', 'scikit-learn',

            # Domains
            'healthcare', 'pharma', 'clinical', 'drug discovery',
            'precision medicine', 'personalized medicine',
            'immunotherapy', 'oncology',

            # Roles
            'scientist', 'engineer', 'architect', 'lead', 'manager',
            'analyst', 'developer', 'researcher'
        ]

        found_keywords = [kw for kw in keywords if kw in text]

        # Also extract from phrases
        skill_patterns = [
            r'\b(python|r|java|scala|sql)\b',
            r'\b(aws|azure|gcp)\b',
            r'\b(docker|kubernetes|spark)\b',
            r'\b(machine learning|deep learning|ai)\b',
            r'\b(rna-?seq|chip-?seq|atac-?seq)\b'
        ]

        for pattern in skill_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            found_keywords.extend([m.lower() for m in matches])

        return list(set(found_keywords))

    def _calculate_relevance_score(
        self,
        file_content: str,
        keywords: List[str]
    ) -> float:
        """Calculate relevance score for a file."""
        file_content_lower = file_content.lower()

        score = 0.0

        for keyword in keywords:
            # Count occurrences
            count = file_content_lower.count(keyword)

            # Weight by keyword importance (simple heuristic)
            weight = 1.0
            if keyword in ['python', 'r', 'machine learning', 'bioinformatics']:
                weight = 2.0
            elif keyword in ['aws', 'docker', 'sql']:
                weight = 1.5

            score += count * weight

        # Normalize by file length
        if len(file_content) > 0:
            score = score / (len(file_content) / 1000)  # per 1000 chars

        return score


In [23]:

# ═══════════════════════════════════════════════════════════════════
# CV GENERATOR V2
# ═══════════════════════════════════════════════════════════════════

class CVGeneratorV2:
    """Generate customized CV files using organized reference files."""

    def __init__(self, llm: LLMProvider, analyzer: OrganizedCVAnalyzer, config: Config):
        self.llm = llm
        self.analyzer = analyzer
        self.config = config

    def generate_cv_files(
        self,
        job_description: str,
        job_title: str,
        company_name: str,
        output_dir: Optional[str] = None
    ) -> Dict[str, str]:
        """Generate all CV files for a job application."""

        print("\n" + "="*70)
        print("🚀 GENERATING CUSTOMIZED CV")
        print("="*70)
        print(f"   Job Title: {job_title}")
        print(f"   Company: {company_name}")

        # Create output directory
        if not output_dir:
            timestamp = datetime.now().strftime("%Y_%m")
            safe_title = re.sub(r'[^\w\s-]', '', job_title).replace(' ', '_')
            safe_company = re.sub(r'[^\w\s-]', '', company_name).replace(' ', '_')
            output_dir = f"{self.config.OUTPUT_BASE_PATH}/{safe_title}_{safe_company}_{timestamp}"

        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        print(f"   Output: {output_path}")

        generated_files = {}

        # Generate each category
        for category in self.config.CV_CATEGORIES:
            print(f"\n{'='*70}")
            print(f"📝 Generating {category.upper()}")
            print(f"{'='*70}")

            # Find relevant reference files
            reference_files = self.analyzer.find_relevant_files(
                job_description=job_description,
                category=category,
                top_k=5
            )

            if not reference_files:
                print(f"   ⚠️  No reference files found for {category}")
                continue

            # Generate content
            content = self._generate_file_content(
                category=category,
                job_description=job_description,
                job_title=job_title,
                company_name=company_name,
                reference_files=reference_files
            )

            if content and "Error:" not in content:
                filename = f"{category}.tex"
                filepath = output_path / filename

                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(content)

                generated_files[filename] = str(filepath)
                print(f"   ✅ Saved: {filename}")
            else:
                print(f"   ❌ Failed: {content}")

        # Generate main resume file
        print(f"\n{'='*70}")
        print("📄 Generating main resume file")
        print(f"{'='*70}")

        resume_content = self._generate_resume_main(job_title, company_name)

        safe_title = re.sub(r'[^\w\s-]', '', job_title).replace(' ', '_')
        resume_filename = f"resume_konstantinos_billis_{safe_title}.tex"
        resume_path = output_path / resume_filename

        with open(resume_path, 'w', encoding='utf-8') as f:
            f.write(resume_content)

        generated_files['main_resume'] = str(resume_path)
        print(f"   ✅ Saved: {resume_filename}")

        # Generate README
        readme_content = self._generate_readme(
            job_title, company_name, job_description
        )

        readme_path = output_path / "README.md"
        with open(readme_path, 'w', encoding='utf-8') as f:
            f.write(readme_content)

        print(f"\n✅ Generated {len(generated_files)} files in: {output_path}")

        return generated_files

    def _generate_file_content(
        self,
        category: str,
        job_description: str,
        job_title: str,
        company_name: str,
        reference_files: List[Dict]
    ) -> str:
        """Generate content for a specific category."""

        # Prepare reference examples
        reference_examples = []
        for i, ref_file in enumerate(reference_files[:5], 1):
            # ref_content = ref_file['content'][:1500]  # Limit length
            ref_content = ref_file['content'][:10500]  # Limit length
            reference_examples.append(
                f"\n### Example {i} (from {ref_file['source_dir']}):\n{ref_content}\n"
            )

        reference_text = '\n'.join(reference_examples)

        # Category-specific instructions
        category_instructions = {
            'summary': "Create a compelling professional summary (3-5 sentences) similar to Summaries of the Reference Examples and based on job adv",

            'experience': "- Use exact titles, names of companies and dates from Reference Examples\
                - Include department/faculty if it clarifies specialization \
                - report in same order using \\cventry commands \
                - and only adjust the \\cvitems of Reference Examples based on the job adv.",

            'key_skills': "Organize skills by 2 or 3 categories - Use exact same names of Reference Examples- using \\cvskill commands as on Reference Examples. \
                Prioritize skills mentioned in the job description and Reference Examples.",
            'education': "Use exact degree names from Reference Examples, List educational background using \\cventry commands. Include relevant coursework or thesis topics if relevant to job.",
            'additional_info': "Include publications, certifications, awards, or other relevant information using \\cvhonor or \\cventry commands, always based on Reference Examples."
        }

        instruction = category_instructions.get(category, "Generate appropriate content for this section.")

        prompt = f"""You are an expert CV writer creating LaTeX files for the Awesome-CV template.

**Task:** Generate a {category}.tex file customized for this job application.

**Job Information:**
- Title: {job_title}
- Company: {company_name}

**Job Description (extract key requirements):**
{job_description[:1500]}

**Reference Examples (from successful CVs):**
{reference_text}

**Instructions:**
1. {instruction}
2. Use ONLY Awesome-CV LaTeX commands:
   - \\cventry{{date}}{{title}}{{company}}{{location}}{{description}}
   - \\cvskill{{category}}{{skills list}}
   - \\cvhonor{{award}}{{organization}}{{location}}{{date}}
3. Use similar keywords to job description, and Tailor content of Reference Examples to match job requirements
4. Keep formatting consistent with Reference Examples
5. DO NOT include "Here is the generated `key_skills.tex` file:", \\documentclass, \\begin{{document}},  \\end{{document}}
6. Return ONLY the LaTeX content, no explanations or markdown. Same format and structure as examples. Only select relevant experiences of the Reference Examples and change content based on job description.
7. In resume tex file, use full path of files
8. Prepare all required files


Generate the {category}.tex content now:
"""

        print(f"   🤖 Generating with {self.config.LLM_PROVIDER}...")

        prefix = "prompt_output"
        filename = f"{prefix}_{category}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(prompt)
        print(f"✅ Saved text to {filename}")



        return self.llm.generate(prompt)

    def _generate_resume_main(self, job_title: str, company_name: str) -> str:
        """Generate the main resume.tex file."""

        return r"""%!TEX TS-program = xelatex
%!TEX encoding = UTF-8 Unicode

\documentclass[11pt, a4paper]{awesome-cv}

\geometry{left=1.4cm, top=1cm, right=1.4cm, bottom=1.2cm, footskip=.5cm}

\fontdir[fonts/]

\colorlet{awesome}{awesome-nephritis}

\renewcommand{\acvHeaderSocialSep}{\quad\textbar\quad}

%-------------------------------------------------------------------------------
%	PERSONAL INFORMATION
%-------------------------------------------------------------------------------
\photo[circle,noedge,right]{/Users/kbillis/bin/Awesome-CV/examples/photos/kbillis_2024_profphoto.jpg}

\name{Konstantinos}{Billis}
\position{PhD in Bioinformatics{\enskip\cdotp\enskip}Data Solutions Developer in Healthcare \& Life Sciences}

\address{Zurich, CH (Permit: B) - Citizenships: Greek (EU), British}
\dateofbirth{Aug 16, 1983}
\email{billis.konstantinos@gmail.com}
\homepage{kbillis.github.io/}
\linkedin{konstantinos-billis-phd-baa23a22}
\googlescholar{YsQgBgUAAAAJ&view_op=list_works&sortby=pubdate}{ }

%-------------------------------------------------------------------------------
\begin{document}
\newpage

\makecvheader[C]

\makecvfooter
  {\today}
  {Konstantinos Billis, PhD~~~·~~~Résumé}
  {\thepage}

%-------------------------------------------------------------------------------
%	CV/RESUME CONTENT
%	Each section is imported separately
%-------------------------------------------------------------------------------
\input{summary.tex}
\input{key_skills.tex}
\input{experience.tex}
\input{education.tex}
\input{additional_info.tex}

\end{document}
"""

    def _generate_readme(self, job_title: str, company_name: str, job_description: str) -> str:
        """Generate README for the CV directory."""
        return f"""# CV for {job_title} at {company_name}

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Using: AI-Powered CV Customizer V2

## Job Description

{job_description[:500]}...

## Files Generated

- `resume_konstantinos_billis_*.tex` - Main resume file
- `summary.tex` - Professional summary
- `experience.tex` - Work experience
- `key_skills.tex` - Key skills
- `education.tex` - Education
- `additional_info.tex` - Publications, awards, etc.

## Compilation
```bash
cd "{os.path.basename(self.config.OUTPUT_BASE_PATH)}"
xelatex resume_konstantinos_billis_*.tex
```

## Customization

Generated using:
- LLM Provider: {self.config.LLM_PROVIDER}
- Reference files from organized CV database
- Keyword matching and relevance scoring

## Next Steps

1. Review all generated files
2. Customize any sections as needed
3. Compile to PDF
4. Proofread final output
"""



In [ ]:
read_adv = False
if read_adv:
  #parse the page with the job description
  # langchain gets the public description of the job
  # make sure we are not behind a proxy
  from langchain_community.document_loaders import WebBaseLoader

  jd_url="https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4314922582"
  print(f"Getting information for job {jd_url}")
  loader = WebBaseLoader(jd_url )
  loader.default_parser="html.parser"
  docs = loader.load()
  # docs[0]


  # from langchain_unstructured import UnstructuredLoader
  # loader = UnstructuredLoader(web_url=jd_url)
  # docs=loader.load()

  description = docs[0].page_content
  # print(f"Job Description\n{description} ")


In [24]:
# ═══════════════════════════════════════════════════════════════════
# MAIN WORKFLOW
# ═══════════════════════════════════════════════════════════════════

def main(job_description: str, job_title: str, company_name: str):
    """Main execution workflow."""

    print("="*70)
    print("📄 AI-POWERED CV CUSTOMIZER V2")
    print("="*70)

    # Initialize
    config = Config()

    print(f"\n⚙️  Configuration:")
    print(f"   LLM Provider: {config.LLM_PROVIDER}")
    print(f"   Organized Files: {config.ORGANIZED_FILES_PATH}")
    print(f"   Output Path: {config.OUTPUT_BASE_PATH}")

    # Initialize LLM
    print(f"\n🤖 Initializing LLM...")
    llm = get_llm_provider(config)

    # Initialize Analyzer
    print(f"\n📚 Loading organized CV files...")
    analyzer = OrganizedCVAnalyzer(config.ORGANIZED_FILES_PATH)
    analyzer.load_all_categories()

    # Generate CV
    generator = CVGeneratorV2(llm, analyzer, config)
    # generator = CVGeneratorV3(llm, analyzer, config)


    generated_files = generator.generate_cv_files(
        job_description=job_description,
        job_title=job_title,
        company_name=company_name
    )

    print("\n" + "="*70)
    print("✅ CV GENERATION COMPLETE!")
    print("="*70)
    print(f"\nGenerated files:")
    for filename, filepath in generated_files.items():
        print(f"   📄 {filename}")

    print("\n💡 Next steps:")
    print("   1. Review generated files")
    print("   2. Compile: xelatex resume_*.tex")
    print("   3. Customize if needed")

# Example usage
if __name__ == "__main__":
    # Example job application
    job_description = """
    Senior Bioinformatics Scientist - Cancer Genomics

    Join our Oncology Research team to lead bioinformatics analysis of
    multi-omics cancer data. The ideal candidate will have extensive
    experience in cancer genomics, RNA-seq analysis, and biomarker discovery.

    Key Responsibilities:
    - Lead bioinformatics analysis of multi-omics cancer data
    - Develop computational pipelines for biomarker discovery
    - Collaborate with wet-lab scientists and clinicians
    - Mentor junior bioinformaticians

    Required Skills:
    - PhD in Bioinformatics, Computational Biology, or related field
    - 5+ years experience in cancer genomics
    - Expert in Python, R, and Unix/Linux
    - Experience with RNA-seq, WGS, WES analysis
    - Strong publication record
    - Knowledge of AWS/cloud computing

    Preferred:
    - Experience with single-cell sequencing
    - Knowledge of immunotherapy biomarkers
    - Machine learning experience
    """

    job_title = "Senior Bioinformatics Scientist"
    company_name = "Roche"

    main(job_description, job_title, company_name)

📄 AI-POWERED CV CUSTOMIZER V2

⚙️  Configuration:
   LLM Provider: ollama
   Organized Files: /content/drive/MyDrive/Documents/work/jobs_work_applications_and_supporting_docs/AI/CVS_awesome_files/
   Output Path: /content/drive/MyDrive/Documents/work/jobs_work_applications_and_supporting_docs/AI/output/

🤖 Initializing LLM...
✅ Ollama initialized on 127.0.0.1:11434 with model: llama3.1:8b

📚 Loading organized CV files...

🔍 Loading organized CV files...
   ✅ additional_info     :   7 files
   ✅ education           :   9 files
   ✅ experience          :  48 files
   ✅ key_skills          :  47 files
   ✅ summary             :  39 files

🚀 GENERATING CUSTOMIZED CV
   Job Title: Senior Bioinformatics Scientist
   Company: Roche
   Output: /content/drive/MyDrive/Documents/work/jobs_work_applications_and_supporting_docs/AI/output/Senior_Bioinformatics_Scientist_Roche_2025_11

📝 Generating SUMMARY
   📋 Top 5 files for summary:
      1. Plants_2025_03                 (score: 83.67181153533711